<a href="https://colab.research.google.com/github/dhrruvchotai/Current_Learnings/blob/main/Pytorch/fashion_mnist_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import kagglehub
path = kagglehub.dataset_download("zalando-research/fashionmnist")

Using Colab cache for faster access to the 'fashionmnist' dataset.


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [19]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

'cuda'

In [20]:
df = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_train.csv")

In [21]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.shape

(60000, 785)

In [23]:
X = df.drop(columns=["label"])
y = df["label"]

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [25]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [26]:
X_train

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
48572,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
38696,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
13611,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
35213,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
31766,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.003922,0.0,...,0.000000,0.000000,0.003922,0.000000,0.043137,0.145098,0.023529,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,0.0,0.0,0.0,0.0,0.0,0.011765,0.027451,0.000000,0.000000,0.0,...,0.682353,0.705882,0.768627,0.921569,0.000000,0.000000,1.000000,0.898039,0.65098,0.0
38158,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
860,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
15795,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007843,0.000000,0.0,...,0.368627,0.290196,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0


In [27]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features.values,dtype=torch.float32)
    self.labels = torch.tensor(labels.values,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index],self.labels[index]

In [28]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [29]:
# train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
# test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [30]:
class Model(nn.Module):
  #in this arch number of neuron in every hidden layer will be the same
  def __init__(self,input_dim,output_dim,num_hidden_layers,neurons_per_layer,dropout_rate):
    super().__init__()
    layers = []

    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dim,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      # layers.append(nn.Dropout(p=0.3))
      layers.append(nn.Dropout(p=dropout_rate))
      input_dim = neurons_per_layer

    #adding output layer
    layers.append(nn.Linear(neurons_per_layer,output_dim))

    #sequential expects all layers individually not in list so passing layers individually by desturcturing the list using *
    self.model = nn.Sequential(*layers)

  def forward(self,X):
    return self.model(X)

In [31]:
def objective(trial):
  #select next hyperparam values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers",1,5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer",8,128,step=8)
  epochs = trial.suggest_int("epochs",10,50,step=10)
  learning_rate = trial.suggest_float("learning_rate",1e-5,1e-1,log=True)
  dropout_rate = trial.suggest_float("dropout_rate",0.1,0.5,step=0.1)
  batch_size = trial.suggest_categorical("batch_size",[16,32,64,128])
  optimizer_name = trial.suggest_categorical("optimizer_name",["Adam","SGD","RMSProp"])
  weight_decay = trial.suggest_float("weight_decay",1e-5,1e-3)


  train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
  test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

  #model init
  input_dim = 784
  output_dim = 10
  model = Model(input_dim,output_dim,num_hidden_layers,neurons_per_layer,dropout_rate)
  model = model.to(device)

  #params init
  # epochs = 50
  # learning_rate = 0.1

  #loss function and optimizer select
  loss_function = nn.CrossEntropyLoss()

  if optimizer_name == "Adam":
    optimizer = optim.Adam(model.parameters(),learning_rate,weight_decay=weight_decay)
  elif optimizer_name == "SGD":
    optimizer = optim.SGD(model.parameters(),learning_rate,weight_decay=weight_decay)
  else:
    optimizer = optim.RMSprop(model.parameters(),learning_rate,weight_decay=weight_decay)

  #training loop
  for epoch in range(epochs):
    for batch_features,batch_labels in train_loader:
      batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
      #forward pass
      y_pred = model(batch_features)
      #cal loss
      loss = loss_function(y_pred,batch_labels)
      #clear gradients
      optimizer.zero_grad()
      #backprop
      loss.backward()
      #update params
      optimizer.step()

  #eval
  model.eval()
  total = 0
  correct = 0

  with torch.no_grad():
    for batch_features, batch_labels in test_loader:
      batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
      outputs = model(batch_features)
      _, predicted = torch.max(outputs,dim=1)
      total = total + batch_features.shape[0]
      correct = correct + (predicted == batch_labels).sum().item()
    accuracy = correct/total
  return accuracy

In [34]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 12.4 MB/s eta 0:00:00


In [35]:
import optuna
study = optuna.create_study(direction="maximize")


[I 2026-02-17 13:26:25,147] A new study created in memory with name: no-name-4b1774b6-d0a9-4540-92db-53ab5041dfee


In [36]:
study.optimize(objective,n_trials=10)

[I 2026-02-17 13:31:33,429] Trial 0 finished with value: 0.8906666666666667 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 120, 'epochs': 50, 'learning_rate': 0.021466892688255905, 'dropout_rate': 0.2, 'batch_size': 16, 'optimizer_name': 'SGD', 'weight_decay': 0.00012559274625551165}. Best is trial 0 with value: 0.8906666666666667.
[I 2026-02-17 13:35:17,256] Trial 1 finished with value: 0.8390833333333333 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 112, 'epochs': 20, 'learning_rate': 1.880585513869681e-05, 'dropout_rate': 0.5, 'batch_size': 16, 'optimizer_name': 'Adam', 'weight_decay': 0.0009359875945938372}. Best is trial 0 with value: 0.8906666666666667.
[I 2026-02-17 13:36:48,901] Trial 2 finished with value: 0.8649166666666667 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 40, 'epochs': 40, 'learning_rate': 0.012845588109079698, 'dropout_rate': 0.1, 'batch_size': 64, 'optimizer_name': 'SGD', 'weight_decay': 0.0005838673064377633}. Be

In [37]:
study.best_value

0.8906666666666667

In [38]:
study.best_params

{'num_hidden_layers': 1,
 'neurons_per_layer': 120,
 'epochs': 50,
 'learning_rate': 0.021466892688255905,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'SGD',
 'weight_decay': 0.00012559274625551165}

In [ ]:
# class Model(nn.Module):

#   def __init__(self,num_features):
#     super().__init__()
#     self.neural_net = nn.Sequential(
#         nn.Linear(num_features,128),
#         nn.BatchNorm1d(128),
#         nn.ReLU(),
#         nn.Dropout(p=0.3),
#         nn.Linear(128,64),
#         nn.BatchNorm1d(64),
#         nn.ReLU(),
#         nn.Dropout(p=0.3),
#         nn.Linear(64,10)
#         #will not add softmax here because there is softmax function in built in cross entropy loss in pytorch
#     )

#   def forward(self,features):
#     return self.neural_net(features)


In [ ]:
# !pip install torchinfo

In [ ]:
# from torchinfo import summary

In [ ]:
# summary(model)

In [ ]:
# num_features = X_train.shape[1]
# model = Model(num_features)
# model = model.to(device)

In [ ]:
# epochs = 100
# learning_rate = 0.1

In [ ]:
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(),learning_rate,weight_decay=1e-4)

In [ ]:
# num_train_batches = len(train_loader)
# num_train_batches

In [ ]:
# #training loop
# for epoch in range(epochs):
#   total_epoch_loss = 0
#   for batch_features,batch_labels in train_loader:
#     batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
#     #forward pass
#     y_pred = model(batch_features)
#     #cal loss
#     loss = loss_function(y_pred,batch_labels)
#     #clear gradients
#     optimizer.zero_grad()
#     #backprop
#     loss.backward()
#     #update params
#     optimizer.step()
#     total_epoch_loss += loss.item()

#   print(f"Epoch {epoch+1} Loss : {total_epoch_loss/num_train_batches}")

In [ ]:
#To Check how torch.max function works
# torch.manual_seed(42)
# outputs = torch.rand(32,10)
# print(outputs[0])

In [ ]:
# _, output = torch.max(outputs,dim=1)
# print(f"{_} and {output}")

In [ ]:
# model.eval()

In [ ]:
# total = 0
# correct = 0

# with torch.no_grad():
#   for batch_features, batch_labels in test_loader:
#     batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
#     outputs = model(batch_features)
#     _, predicted = torch.max(outputs,dim=1)
#     total = total + batch_features.shape[0]
#     correct = correct + (predicted == batch_labels).sum().item()
# print(correct/total)

In [ ]:
# total = 0
# correct = 0

# with torch.no_grad():
#   for batch_features, batch_labels in train_loader:
#     batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
#     outputs = model(batch_features)
#     _, predicted = torch.max(outputs,dim=1)
#     total = total + batch_features.shape[0]
#     correct = correct + (predicted == batch_labels).sum().item()
# print(correct/total)